<a href="https://colab.research.google.com/github/dipayandas97/Deep-Learning-Notebooks/blob/master/BayesianCNN_NIH_sampleDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from google.colab import drive
import os
import torch

drive.mount('/content/drive/')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#Dataset

In [2]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from PIL import Image
import csv

datapath = '/content/drive/My Drive/ISI/COVID-19/Bayesian_CNN/NIH_sample_dataset/images/'
labelpath = '/content/drive/My Drive/ISI/COVID-19/Bayesian_CNN/NIH_sample_dataset/sample_labels.csv'
list_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.Grayscale(num_output_channels=3),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[130], std=[63])])

class NIH_sample_dataset(Dataset):
    def __init__(self, datapath, labelpath, transforms=None):
        
        self.datapath = datapath
        self.labelpath = labelpath
        self.transforms = transforms
        self.list_of_labels = [ 'Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass',
                                'Nodule','Pneumonia','Pneumothorax','Consolidation','Edema',
                                'Emphysema', 'Fibrosis','Pleural_Thickening', 'Hernia','No Finding']

        #build list of (name,label) tuples as self.labels
        label_file = pd.read_csv(self.labelpath)
        image_names = label_file['Image Index']
        labels = label_file['Finding Labels']
        self.labels = []
        for i in range(image_names.shape[0]):
            self.labels.append((image_names[i], self.oneHotEncode(labels[i])))
        
    def oneHotEncode(self, label):
        encoded_label = torch.zeros(15)
        for idx,name in enumerate(self.list_of_labels):
            if name in label:
                encoded_label[idx] = 1.
        assert label.count('|')+1 == torch.sum(encoded_label).item()
        return encoded_label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_name, image_label = self.labels[idx]
        image_path = os.path.join(self.datapath,image_name)
        image = Image.open(open(image_path, 'rb'))

        #apply transforms
        if self.transforms:
            image = self.transforms(image)

        return ((image, image_label))

def get_dataloader(dataset, test_size=0.2, batch_size=32, num_workers=1):    #Expects Dataset class object
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    split_idx = int(len(dataset) * test_size)

    train_ids, test_ids = indices[split_idx:], indices[:split_idx]
    train_sampler = SubsetRandomSampler(train_ids)
    test_sampler = SubsetRandomSampler(test_ids)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler, num_workers=num_workers)

    return train_loader, test_loader    

In [4]:
dataset = NIH_sample_dataset(datapath=datapath, labelpath=labelpath, transforms=list_transforms)
train_loader, test_loader = get_dataloader(dataset=dataset, test_size=0.2, batch_size=32, num_workers=6)

#Model

In [3]:
import torch
!git clone https://github.com/kumar-shridhar/PyTorch-BayesianCNN

os.chdir('/content/PyTorch-BayesianCNN/')
os.getcwd()

from torch.optim import Adam, lr_scheduler
from torch.nn import functional as F

import data
import utils
import metrics
import config_bayesian as cfg

fatal: destination path 'PyTorch-BayesianCNN' already exists and is not an empty directory.


In [4]:
import torch.nn as nn
import math
from layers import BBB_Linear, BBB_Conv2d
from layers import BBB_LRT_Linear, BBB_LRT_Conv2d
from layers import FlattenLayer, ModuleWrapper

class get_model(ModuleWrapper):
    def __init__(self, outputs=2, inputs=3, priors=None, layer_type='lrt', activation_type='relu'):
        super(get_model, self).__init__()

        self.num_classes = outputs
        self.layer_type = layer_type
        self.priors = priors

        if layer_type=='lrt':
            BBBLinear = BBB_LRT_Linear
            BBBConv2d = BBB_LRT_Conv2d
        elif layer_type=='bbb':
            BBBLinear = BBB_Linear
            BBBConv2d = BBB_Conv2d
        else:
            raise ValueError("Undefined layer_type")
        
        if activation_type=='softplus':
            self.act = nn.Softplus
        elif activation_type=='relu':
            self.act = nn.ReLU
        else:
            raise ValueError("Only softplus or relu supported")

        self.conv1 = BBBConv2d(inputs, 64, 5, stride=5, padding=1, bias=True, priors=self.priors)
        self.act1 = self.act()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = BBBConv2d(64, 192, 5, padding=2, bias=True, priors=self.priors)
        self.act2 = self.act()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = BBBConv2d(192, 384, 3, padding=1, bias=True, priors=self.priors)
        self.act3 = self.act()

        self.conv4 = BBBConv2d(384, 256, 3, padding=1, bias=True, priors=self.priors)
        self.act4 = self.act()

        self.conv5 = BBBConv2d(256, 128, 3, padding=1, bias=True, priors=self.priors)
        self.act5 = self.act()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_1 = nn.BatchNorm2d(128)

        self.conv6 = BBBConv2d(128, 128, 3, padding=1, bias=True, priors=self.priors)
        self.act6 = self.act()

        self.conv7 = BBBConv2d(128, 128, 3, padding=0, bias=True, priors=self.priors)
        self.act7 = self.act()

        self.conv8 = BBBConv2d(128, 128, 3, padding=0, bias=True, priors=self.priors)
        self.act8 = self.act()
        
        self.flatten = FlattenLayer(1 * 1 * 128)

        self.fc1 = BBBLinear(1*1*128, 64, bias=True, priors=self.priors)
        self.act9 = self.act()
        
        self.fc2 = BBBLinear(64, 32, bias=True, priors=self.priors)
        self.act10 = self.act()
        
        self.fc3 = BBBLinear(32, outputs, bias=True, priors=self.priors)
        self.act11 = nn.Sigmoid()

    #def forward_(self,x):
    #    print('Own forward()')
        
model = get_model(outputs=15, inputs=3).to(device)

#Train/Test Routine

In [5]:
#Train routine
from tqdm.notebook import tqdm as tqdm
from sklearn.model_selection import train_test_split

class ELBO(nn.Module):
    def __init__(self, train_size):
        super(ELBO, self).__init__()
        self.train_size = train_size

    def forward(self, input, target, kl, beta):
        assert not target.requires_grad
        loss = nn.BCELoss()
        return loss(input, target) + beta * kl

def acc(outputs, targets):
    #Thresholding : 0.5
    outputs[torch.where(outputs>0.5)] = 1.
    outputs[torch.where(outputs!=1.)] = 0.
    return torch.mean((outputs==targets).float(), dim=0).detach().cpu().numpy()

def train_model(net, dataloader, optimizer, criterion, num_ens=1, beta_type=0.1, epoch=None, num_epochs=None):
    net.train()
    training_loss = 0.0
    accs = []
    kl_list = []

    for i,(X, Y) in tqdm(enumerate(dataloader,1), total=len(dataloader)):

        optimizer.zero_grad()

        inputs, labels = X.to(device), Y.to(device)
        outputs = torch.zeros(X.size(0), net.num_classes).to(device)
        
        net_out, kl = net(inputs)
        outputs[:, :] = net_out

        kl_list.append(kl.item())

        beta = metrics.get_beta(i-1, len(dataloader), beta_type, epoch, num_epochs)
        loss = criterion(outputs, labels, kl, beta)
        loss.backward()
        optimizer.step()

        accs.append(acc(outputs.data, labels.data))
        training_loss += loss.cpu().data.numpy()
        break
    del X, Y
    return training_loss/len(dataloader), np.mean(np.asarray(accs), axis=0), np.mean(kl_list)

def validate_model(net, dataloader, criterion, batch_size = 32, num_ens=1, beta_type=0.1, epoch=None, num_epochs=None):
    net.train()
    valid_loss = 0.0
    accs = []
    kl_list = []

    for i,(X,Y) in tqdm(enumerate(dataloader,1), total=len(dataloader)):
    
        inputs, labels = X.to(device), Y.to(device)
        outputs = torch.zeros(X.size(0), net.num_classes).to(device)

        net_out, kl = net(inputs)
        outputs[:, :] = net_out

        kl_list.append(kl.item())

        beta = metrics.get_beta(i-1, len(dataloader), beta_type, epoch, num_epochs)
        valid_loss += criterion(outputs, labels, kl, beta).item()
        accs.append(acc(outputs, labels))
    
    del X, Y
    return valid_loss/len(dataloader), np.mean(np.asarray(accs), axis=0), np.mean(kl_list)

In [6]:
# Hyper Parameter settings
layer_type = 'bbb'
activation_type = 'relu'
priors = cfg.priors

train_ens = cfg.train_ens
valid_ens = cfg.valid_ens
n_epochs = 500
lr_start = cfg.lr_start
num_workers = cfg.num_workers
valid_size = cfg.valid_size
batch_size = cfg.batch_size
beta_type = cfg.beta_type

dataset = NIH_sample_dataset(datapath=datapath, labelpath=labelpath, transforms=list_transforms)
train_loader, test_loader = get_dataloader(dataset=dataset, test_size=0.2, batch_size=32, num_workers=0)

criterion = ELBO(len(train_loader)).to(device)
optimizer = Adam(model.parameters(), lr=0.001)
lr_sched = lr_scheduler.ReduceLROnPlateau(optimizer, patience=6, verbose=True, factor=0.5)

path = '/content/drive/My Drive/ISI/COVID-19/Bayesian_CNN/best_model_NIHsamplesData_state_dict.pt'
#model = get_model()
#model.load_state_dict(torch.load(path))

train_loss_list, test_loss_list = [], []

for epoch in range(n_epochs):  
    print('Epoch :',epoch,'/',n_epochs)

    train_loss, train_acc, train_kl = train_model(model, train_loader, optimizer, criterion, num_ens=train_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    test_loss, test_acc, test_kl = validate_model(model, test_loader, criterion, num_ens=valid_ens, beta_type=beta_type, epoch=epoch, num_epochs=n_epochs)
    lr_sched.step(test_loss)

    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)

    print('Train Loss:',train_loss, '| Train KL:',train_kl)
    print('Train Acc:', train_acc)
    print('Test Loss:',test_loss,'| Test KL:',test_kl)
    print('Test Acc:', test_acc)

    torch.save(model.state_dict(), path)

    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(train_loss_list, label='Train Loss')
    plt.plot(test_loss_list, label='Test Loss')
    plt.legend()
    plt.show()

    print('---------------------------------------------------------------------------------')

Epoch : 0 / 500


KeyboardInterrupt: ignored